In [ ]:
import urllib.parse
import pandas as pd
import pytz,os
import requests
import json ,requests
from  datetime import datetime,timedelta,time
TIME_ZONE = pytz.timezone('Asia/Kolkata')

In [ ]:
fileUrl ='https://assets.upstox.com/market-quote/instruments/exchange/complete.csv.gz'
symboldf = pd.read_csv(fileUrl)
symboldf['expiry'] = pd.to_datetime(symboldf['expiry']).apply(lambda x: x.date())   
symboldf = symboldf[symboldf.exchange == 'NSE_EQ']
eqCsv = pd.read_csv('EQUITY_L.csv')
isinList = 'NSE_EQ|' + eqCsv[' ISIN NUMBER']
symboldf = symboldf[symboldf.instrument_key.isin(isinList)] 
symboldf

In [ ]:
#fromDate = (datetime.now(TIME_ZONE)  - timedelta(days=10000)) .strftime("%Y-%m-%d")
#fromDate

In [ ]:
def getHistoricalData(symInfo):
    res = None
    try:
        parseInstrument = urllib.parse.quote(symInfo.instrument_key)
        fromDate = (datetime.now(TIME_ZONE)  - timedelta(days=10000)) .strftime("%Y-%m-%d")
        todate = datetime.now(TIME_ZONE).strftime("%Y-%m-%d")
        url = f'https://api.upstox.com/v2/historical-candle/{parseInstrument}/day/{todate}/{fromDate}' 
        #print(url)
       
        res = requests.get(url,headers={'accept': 'application/json',},params={},timeout=5.0)
        candleRes = res.json()
        
        if 'data' in candleRes and 'candles' in candleRes['data'] and  candleRes['data']['candles']:
            candleData = pd.DataFrame(candleRes['data']['candles'])
            candleData.columns = ['date','open','high','low', 'close','vol','oi']
            candleData['date'] = pd.to_datetime(candleData['date']).dt.tz_convert('Asia/Kolkata')
            candleData['symbol'] = symInfo.tradingsymbol
            print(symInfo.tradingsymbol, len(candleData))
            return candleData
        else:
            print('No data',symInfo.instrument_key, candleRes)
    except :
        print(f'Error in data fetch for {symInfo.instrument_key} {res} {e}')

In [ ]:
#getHistoricalData(symboldf.iloc[1])

In [ ]:
candledfList = []
for i in symboldf.index[:2]:
    res = getHistoricalData(symboldf.loc[i])
    if res is not None: candledfList.append(res) 



In [ ]:
finalDataDf = pd.concat(candledfList,ignore_index= True)
finalDataDf

In [ ]:
isCsv = True
for symData in candledfList:
    try:
        filename = symData.iloc[0]['symbol']
        if isCsv : 
            filename = f'{filename}.csv'
            symData.to_csv(filename,index=False)
        else:
            filename = f'{filename}.parquet' 
            symData.to_parquet(filename , engine='pyarrow')
    except Exception as e:
        print(f'Error {e}')

In [ ]:
pd.read_parquet('C:\\Users\\eqEOD\\20MICRONS.parquet')